In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px


In [61]:
df = pd.read_pickle('log_dw.pkl')

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15249 entries, 0 to 15429
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Ip                   15249 non-null  object        
 1   Data                 15249 non-null  datetime64[ns]
 2   Metodo               15249 non-null  object        
 3   URL                  15249 non-null  object        
 4   Protocolo            15249 non-null  object        
 5   Status               15249 non-null  int64         
 6   E_Mobile             15249 non-null  bool          
 7   E_Tablet             15249 non-null  bool          
 8   E_Pc                 15249 non-null  bool          
 9   E_Bot                15249 non-null  bool          
 10  Navegador            15249 non-null  object        
 11  Sistema_Operacional  15249 non-null  object        
 12  Continente           15249 non-null  object        
 13  Pais                 15249 non-null 

In [63]:
pd.set_option('display.max_columns', None) # maximo de colunas

## Informação sobre o dataset

In [64]:

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15249 entries, 0 to 15429
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Ip                   15249 non-null  object        
 1   Data                 15249 non-null  datetime64[ns]
 2   Metodo               15249 non-null  object        
 3   URL                  15249 non-null  object        
 4   Protocolo            15249 non-null  object        
 5   Status               15249 non-null  int64         
 6   E_Mobile             15249 non-null  bool          
 7   E_Tablet             15249 non-null  bool          
 8   E_Pc                 15249 non-null  bool          
 9   E_Bot                15249 non-null  bool          
 10  Navegador            15249 non-null  object        
 11  Sistema_Operacional  15249 non-null  object        
 12  Continente           15249 non-null  object        
 13  Pais                 15249 non-null 

# ---------- KPIs Principais -------------------
- Total de reuisições
- Ips únicos
- Taxa de erros (4xx/5xx)
- URL mais visitada

In [65]:
# --- KPIs Principais ---

#total de requesições
total = df.URL.value_counts().sum()
print(f'Total de requesições: {total:,}')
# verificar o total de ips unicos
ips_unicos = df.Ip.nunique() 
print(f'Total de ips unicos: {ips_unicos}')

# url mais 
s_a = df.URL.groupby(df.URL).count().sort_values(ascending=False)
print(f'O url mais visitado: {s_a.keys()[0]}')

#Taxa de erros(4xx/5xx)
total_req = len(df)
erros_req = len(df[df.Status  >= 400])
error_rate = (erros_req / total_req) * 100 if total_req > 0 else 0
print(f"Taxa de Erros (4xx/5xx): {error_rate:.2f}%")


Total de requesições: 15,249
Total de ips unicos: 845
O url mais visitado: /filter
Taxa de Erros (4xx/5xx): 3.44%


Formula da taxa de erro:
$Taxa deerro = (Número de Erros/Total de tentativas) * 100%% $

#  Passo1 : ver recursos mais acessados

In [66]:
# Analise 1: recursos mais acessados
# top 10
recursos_mais_acess = df.URL.groupby(df.URL).count().sort_values(ascending=False).reset_index(name = 'Numero de vezes que foi acessado')[:10]
recursos_mais_acess

,URL,Numero de vezes que foi acessado
0,/filter,608
1,/m/filter,164
2,,163
3,/image/33888,148
4,/image/11947,115
5,/customer/register,93
6,/faviconico,74
7,/apple-touch-icon-precomposedpng,42
8,/login/auth,39
9,/search,39


In [67]:
# Mostrando graficamente
fig = px.bar(data_frame= recursos_mais_acess, y = 'URL', x = 'Numero de vezes que foi acessado',title = 'Top 10 dos Recursos mais acessados', color = 'URL')
fig.update_yaxes(
    categoryorder="total ascending"
)
fig.show()

# Passo2: destribuição de StatusCode

In [68]:

df.rename(columns = {'Status': 'status_code'}, inplace = True)

In [69]:
code = df.status_code.groupby(df.status_code).count().sort_index().reset_index(name = 'Quantidade')
code

,status_code,Quantidade
0,200,13121
1,301,478
2,302,1121
3,304,4
4,400,3
5,403,6
6,404,499
7,499,10
8,500,1
9,502,5


In [94]:
# # destribuição de estatus
statusCode = df['status_code'].value_counts().reset_index()
statusCode.columns = ['Status', 'Quantidade']

px.pie(statusCode, values = 'Quantidade', names = 'Status', title='Destribução de Status code', hole = 0.4).show()

# Passo3: Ver os horários de pico de acesso

In [71]:
df.columns

Index(['Ip', 'Data', 'Metodo', 'URL', 'Protocolo', 'status_code', 'E_Mobile',
       'E_Tablet', 'E_Pc', 'E_Bot', 'Navegador', 'Sistema_Operacional',
       'Continente', 'Pais', 'Codigo_Pais', 'Regiao', 'Cidade', 'Latitude',
       'Longitude', 'Isp', 'Organizacao', 'As', 'Proxy', 'Hospedagem',
       'Consulta'],
      dtype='object')

In [72]:

df.Data = pd.to_datetime(df.Data) # fromatando o tempo

In [73]:
df.Data = df.Data.dt.tz_localize(None) # type:ignore isso é para normalizar o fusohorario, ou seja, remover.

In [74]:
#acahar os horários de pico de acesso
df['hora'] = df.Data.dt.hour

In [75]:
horaios_picos = df.hora.value_counts().reset_index(name = 'Total de acessos')
horaios_picos

,hora,Total de acessos
0,4,7901
1,5,6710
2,3,638


In [76]:
df["dia_semana"] = df["Data"].dt.day_name()
df.dia_semana = 'Terça-feira' # padronizando...como todas as requisições foram feitas na terça então troque a palavra que estava em ingles para portugues
print(f'Os horarios de picos acessos vão de 0{horaios_picos.hora[0]}-0{horaios_picos.hora[1]}h. Todas foram feitas na {df.dia_semana[0]}' )
fig = px.bar(data_frame=horaios_picos, x = 'hora', y = 'Total de acessos', color = 'hora', title = 'Horarios de Pico de acessos na Terça-feria')
fig.show()

Os horarios de picos acessos vão de 04-05h. Todas foram feitas na Terça-feira


In [92]:
hora_picos = df.Data.dt.floor('h').value_counts().reset_index(name = 'Requesições')
# hora_picos
px.line(data_frame = hora_picos, x="Data", y="Requesições",title='Volume de Requisições por Hora',markers=True)

# organizando a tabela

In [78]:

df = df[['Ip', 'Data','hora', 'dia_semana', 'Metodo', 'URL', 'Protocolo', 'status_code', 'Navegador', 'Sistema_Operacional'
       , 'E_Mobile', 'E_Tablet', 'E_Pc', 'E_Bot', 'Pais', 'Codigo_Pais', 'Regiao', 'Cidade',
       'Latitude', 'Longitude', 'Isp', 'Organizacao']]

# Passo4 - Métodos mais ultilizados /menos utilizados

In [79]:
#
s_a = df.URL.groupby(df.URL).count().sort_values(ascending=False)
print(f'O url mais acessado: {s_a.keys()[0]}\nNúmero de vezes:{s_a.values[0]}')

O url mais acessado: /filter
Número de vezes:608


In [80]:
#
methodo_acsess_max = df.Metodo.groupby(df.Metodo).count().sort_values(ascending=False) # vendo o metodo mais acessado
methodo_acsess_min = df.Metodo.groupby(df.Metodo).count()[df.Metodo.groupby(df.Metodo).count() == df.Metodo.groupby(df.Metodo).count().min()] # vendo o metodo menos acessado
print(f'O método mais acessado é: {methodo_acsess_max.keys()[0]} --> Numero de vezes: {methodo_acsess_max.values[0]}')
print(f'O método menos acessado é: {methodo_acsess_min.keys()[0]} --> Numero de vezes: {methodo_acsess_min.values[0]}')

O método mais acessado é: GET --> Numero de vezes: 15239
O método menos acessado é: HEAD --> Numero de vezes: 10


In [81]:
#Mostrando graficamente
metodos_http = df.Metodo.value_counts().reset_index()
metodos_http.columns = ['Métodos','Quantidade']
px.bar(metodos_http, x = 'Métodos', y ='Quantidade', color = 'Métodos', title = 'Uso de Métodos HTTP')
# df.Metodo.value_counts()

In [82]:
# 3 - IP que fez mais requisições
ip_req_max = df.Ip.groupby(df.Ip).count()[df.Ip.groupby(df.Ip).count() == df.Ip.groupby(df.Ip).count().max()] 
print(f'IP que fez mais requisições: {ip_req_max.keys()[0]}')

IP que fez mais requisições: 66.249.66.194


# Desitribuição por dispositivo

In [83]:
# # 4. Quantidade de requisições feita por cada dispositivo onde está com True
pc = df.E_Pc.groupby(df.E_Pc).count()[df.E_Pc.groupby(df.E_Pc).count() == df.E_Pc.groupby(df.E_Pc).count().min()] 
phone = df.E_Mobile.groupby(df.E_Mobile).count()[df.E_Mobile.groupby(df.E_Mobile).count() == df.E_Mobile.groupby(df.E_Mobile).count().min()]  
tablet = df.E_Tablet.groupby(df.E_Tablet).count()[df.E_Tablet.groupby(df.E_Tablet).count() == df.E_Tablet.groupby(df.E_Tablet).count().min()] 
bot = df.E_Bot.groupby(df.E_Bot).count()[df.E_Bot.groupby(df.E_Bot).count() == df.E_Bot.groupby(df.E_Bot).count().max()]


print(f'''Quantidade de requisições feita por cada dispositivo:
        Tablet: {tablet.values[0]};
        Telefone: {phone.values[0]};
        Pc: {pc.values[0]};
      
        ''')

dispositivos = ['Pc', 'Mobile', 'Tablet', 'Bot']
Quantidades = [pc.values[0], phone.values[0], tablet.values[0], bot.values[0]]
dic_dispositivos = {
 'Dispositivos': dispositivos,
 'Quantidade': Quantidades
}
df_dispositivos = pd.DataFrame(dic_dispositivos)
px.pie(df_dispositivos, values = 'Quantidade', names = 'Dispositivos', title = 'Distribuição por Dispositivos')

Quantidade de requisições feita por cada dispositivo:
        Tablet: 855;
        Telefone: 2650;
        Pc: 2449;

        


In [84]:
# 5. Browser mais utilizado
brower_max = df.Navegador.groupby(df.Navegador).count()[df.Navegador.groupby(df.Navegador).count() == df.Navegador.groupby(df.Navegador).count().max()]  # brower mais usado
print(f'Browser mais utilizado: {brower_max.keys()[0]}')

Browser mais utilizado: Googlebot


In [85]:
navegadores_counts = df.Navegador.value_counts().reset_index().head(10)
navegadores_counts.columns = ['Navegador', 'Uso']
px.pie(navegadores_counts, values = 'Uso', names = 'Navegador', title='Top 10 dos Navegador usados', hole = 0.4)


In [86]:
# sistemas operacionais usados
s_o = df.Sistema_Operacional.value_counts().reset_index()
s_o.columns =['Sistemas Operacionais', 'Uso']
fig = px.bar(s_o, x = 'Sistemas Operacionais', y = 'Uso', color = 'Sistemas Operacionais', title = 'Sistemas Operacionais')
fig.show()

In [87]:
#Mapa de Calor de Acessos
loc = df.groupby(['Pais','Latitude','Longitude']).size().reset_index(name = 'Acessos') # a ideia é saber quandas fezes os valores da latitude e da longitude aparecem no dataset
loc
# quantidade de ocorrências por coordenada

,Pais,Latitude,Longitude,Acessos
0,Canada,43.6532,-79.3832,1
1,China,31.2304,121.4740,2
2,China,38.0360,114.4700,2
3,China,39.9042,116.4070,10
4,Finland,60.1520,24.8876,4
...,...,...,...,...
144,United States,45.5260,-122.9874,2
145,United States,47.2330,-119.8520,2577
146,United States,47.4902,-122.3004,10
147,Uzbekistan,41.2615,69.2177,1


In [88]:
df.Pais.groupby(df.Pais).count()

Pais
Canada                      1
China                      14
Finland                    57
France                   1145
Germany                  1052
Hong Kong                  36
Iran                     1893
Ireland                   115
Japan                       1
Mexico                      1
Morocco                     4
Netherlands               166
Romania                     1
Russia                      9
Singapore                   1
Slovenia                    1
South Africa                1
Spain                       6
Taiwan                      1
The Netherlands             1
United Arab Emirates        1
United Kingdom             30
United States           10710
Uzbekistan                  1
Venezuela                   1
Name: Pais, dtype: int64

In [89]:
fig_map = px.scatter_map(loc, lat='Latitude', 
                            lon='Longitude', 
                            size = 'Acessos', 
                            color = 'Acessos', 
                            title="Mapa de Calor de Acessos",
                            hover_name='Acessos',
                              zoom = 0,
                          
                            )
# fig_map.update_layout(margin={"r":0,"t":40,"l":0,"b":0})   
fig_map.show()

In [90]:
# Acessos por país
pais_Acess = df.value_counts('Pais').reset_index(name = 'Acessos')
# pais_Acess
fig_pa = px.bar(pais_Acess, x = 'Acessos',y = 'Pais',title = 'Acessos por Países')
fig_pa.update_yaxes(
    categoryorder="total ascending"
)
fig_pa

In [91]:
# IP mais ativos
top10_ip = df.value_counts('Ip').reset_index(name = 'Requisições').head(10)
print('Top 10 dos Ips (clientes mais ativos)')
top10_ip

Top 10 dos Ips (clientes mais ativos)


,Ip,Requisições
0,66.249.66.194,3806
1,66.249.66.91,2332
2,40.77.167.170,923
3,91.99.72.15,903
4,207.46.13.136,882
5,157.55.39.245,773
6,130.185.74.243,660
7,207.46.13.9,543
8,17.58.102.43,349
9,40.77.167.13,296
